In [2]:
# 필요한 module import
import tensorflow as tf
import pandas as pd
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
## 1. Data Loading
mnist = pd.read_csv("./data/digitrecognizer/train.csv")
num_of_train = int(mnist.shape[0]*0.7)
train_data = mnist.loc[:num_of_train,:]
test_data = mnist.loc[num_of_train+1:,:]

train_x_data = train_data.drop("label",axis=1,inplace=False)
train_y_data = tf.one_hot(train_data["label"],10)
sess = tf.Session()
train_y_data = sess.run(train_y_data)

test_x_data = test_data.drop("label",axis=1,inplace=False)
train_y_data.shape

(29400, 10)

In [4]:
## 2. Model 정의 (Tensorflow graph 생성)
tf.reset_default_graph() # tensorflow graph 초기화
## 2.1 Placeholder
X = tf.placeholder(shape = [None,784], dtype = tf.float32)
Y = tf.placeholder(shape = [None,10], dtype = tf.float32)
drop_rate = tf.placeholder(dtype = tf.float32)

## 2.2 Convolution
## CNN은 이미지 학습에 최적화 된 Deep learning 방법
## 입력받은 이미지의 형태가 4차원 matrix
## ( 이미지의 개수, 이미지의 width, 이미지의 height, color 수 )
X_img = tf.reshape(X,[-1,28,28,1])

## 2.3 Convolution Layer 1
'''
## filter의 shape = ( width, height, color 수, filter 수 )
filter1 = tf.Variable(tf.random_normal([3,3,1,32]))
## filter를 이용해서 Convolution image 생성
L1 = tf.nn.conv2d(X_img,filter1,strides=[1,1,1,1], padding = "SAME")

# 문제 : sigmoid는 0~1 사이의 실수로 값이 변화해서
## 과정을 반복하다 보면 값이 0으로 수렴함

## 해결 : 만들어진 Convolution에 relu 적용
#L1 = tf.nn.relu(L1)
'''
# 위의 주석들이 아래 코드 한 줄로 정의 가능
L1 = tf.layers.conv2d(inputs=X_img,
                      filters = 32,
                      kernel_size=[3,3],
                      padding= "SAME",
                      strides=1,
                      activation=tf.nn.relu)

## pooling 작업 (resize, sampling 작업) => Optional
# L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1], strides = [1,2,2,1],
#                    padding = "SAME")

# tf.layers.max_pooling2d() 로 변경
L1 = tf.layers.max_pooling2d(inputs= L1,
                             pool_size=[2,2],
                             strides = 2,
                             padding = "SAME")

## 2.4 Convolution Layer 2
L2 = tf.layers.conv2d(inputs=L1,
                      filters = 64,
                      kernel_size=[3,3],
                      padding= "SAME",
                      strides=1,
                      activation=tf.nn.relu)

L2 = tf.layers.max_pooling2d(inputs= L2,
                             pool_size=[2,2],
                             strides = 2,
                             padding = "SAME")

print(L2.shape) # (?, 7, 7, 64) => 7 x 7 크기의 img가 64개 있음

## convolution 결과 : 28 x 28 크기의 원본 img가 
## 7 x 7 크기의 img 64개로 변환

## 2.5 Neural Network

## Convolution의 결과(4차원)를
## Neural network의 입력(2차원)으로
## 사용하기 위해 shape 변경
L2 = tf.reshape(L2, [-1,7*7*64])

# 1번째 layer
# logistic 256개
W1 = tf.get_variable("weight1",shape = [7*7*64,256],initializer=tf.contrib.layers.xavier_initializer())

b1 = tf.Variable(tf.random_normal([256]),name = "bias1")
_layer1 = tf.nn.relu(tf.matmul(L2,W1)+ b1)
layer1 = tf.layers.dropout (_layer1, rate = drop_rate)

# 2번째 layer
W2 = tf.get_variable("weight2",shape = [256,10],initializer=tf.contrib.layers.xavier_initializer())

b2 = tf.Variable(tf.random_normal([10]),name = "bias2")

logits = tf.matmul(layer1,W2)+ b2
H = tf.nn.relu(logits)

## cost 함수
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
## train 
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
train = optimizer.minimize(cost)

# session & 초기화 
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# epoch & batch size
training_epoch = 10
batch_size = 100

# training
for step in range(training_epoch):
    num_of_iteration = int(train_data.shape[0] / batch_size)
    cost_val = 0
    
    for i in range(num_of_iteration):
        batch_x, batch_y = train_x_data[i*batch_size:(i+1)*batch_size],train_y_data[i*batch_size:(i+1)*batch_size]
        _, cost_val = sess.run([train, cost], feed_dict={X: batch_x, Y: batch_y, drop_rate: 1.0})

    if step %5 == 0:
        print(cost_val)
        
#predict check
predict = tf.argmax(H,1)
# result = sess.run(predict, feed_dict={X:test_x_data, drop_rate: 1.0})
# df = pd.DataFrame({
#     'ImageId': [i for i in range(1,28001)],
#     'Label': result
# })
# df.to_csv('./data/digitrecognizer/submission.csv', index=False)

correct = tf.equal(predict, tf.math.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print("Accuracy: {}".format(sess.run(accuracy, feed_dict = {X: train_x_data, Y: train_y_data, drop_rate: 1.0})))

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
(?, 7, 7, 64)
Instructions for updating:
Use keras.layers.dropout instead.
0.040531263
0.0018923819
Accuracy: 0.9931292533874512


In [6]:
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(H, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy,
                            feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))


AttributeError: 'DataFrame' object has no attribute 'test'

In [ ]:
## 결국 우리 MNIST 예제는 입력한 이미지 1개에 대해 예측한 결과가
# H의 값으로 도출
# [0.5, 0.8, 0.99, 0.12, 0.34, ...] 총 10개

## 앙상블은 이런 model이 여러개 있음
## H1 => [0.5, 0.8, 0.99, 0.12, 0.34, ...] 
## H2 => [0.45, 0.38, 0.9, 0.1, 0.7, ...] 
## H3=> [0.11, 0.2 0.19, 0.91, 0.47, ...]
## H3=> [0.3, 0.32 0.3, 0.7, 0.4, ...]

## H의 행별로 다 더함
## SUM => [1.36, 0.8, 1.3, ..]
## 최종 prediction은 SUM한 결과값을 가지고 예측

## 장점 : 정확도 2% 증가
## 단점 : Model 갯수만큼 시간 증가 ex) H 5개 -> 시간 5배 걸림